#### Função para formatação de strings

In [2]:
from unicodedata import normalize
import copy

# Função que formata string desconsiderando maiúsculas e minúsculas e que remove acentos
def formataString(string):
    string = normalize('NFKD', string).encode('ASCII', 'ignore').decode('ASCII')
    return string.upper()

## Definição da estutura que vai formar as árvores TRIE para consulta

In [3]:
class TrieNode(object):

    
    def __init__(self, char: str):   # __init__ é um método especial para fazer construtores
        self.char = char   # caractere do nodo atual
        self.filhos = []   # nodos filhos
        self.pFinalizada = False   # se é o último nodo e a palavra terminou
        self.indices = []   # lista vazia para nodos que não são término de palavra
        ### não é um índice único e sim uma lista, pois podem existir títulos de música repetidos com mais de um artista

#### Operação que adiciona uma string na árvore trie

In [4]:
def addString(raiz, palavra: str, indice: int):   # função usada para adicionar uma palavra nova à estrutura trie
    
    palavra = formataString(palavra)   # formata string de entrada para o padrão (todas as letras maiúsculas sem acento)
    nodo = raiz
    for char in palavra:
        encontradoEmFilho = False
        # busca pelo caractere nos filhos do nodo atual
        for filho in nodo.filhos:
            if filho.char == char: 
                nodo = filho   # apontamos o nodo para o filho que contém esse char
                encontradoEmFilho = True
                break
        
        if not encontradoEmFilho:   # se o caractere não foi encontrado, adiciona novo filho
            novoNodo = TrieNode(char)
            nodo.filhos.append(novoNodo)
            nodo = novoNodo   # apontamos, então, o nodo para seu novo filho e continuamos a iteração
    
    nodo.pFinalizada = True   # indica que até ali pode ser uma palavra (nome de música/artista completo)
    nodo.indices.append(indice)   # assinala o indice passado

#### Operação que procura uma string na árvore trie

In [5]:
def findString(select, dados, raiz, palavra: str) -> (bool, []):
    """
      1. Se a palavra existe e tem algum índice associado, retorna verdadeiro e a lista de índices dela
      2. Se a palavra não existe, retorna falso e uma lista com sugestões de palavras
      
      Sendo o parâmetro select utilizada para indicar se a busca é por artista ou música (0 música, 1 artista)
    """
    palavra = formataString(palavra)   # formata string de entrada para o padrão (todas as letras maiúsculas sem acento)
    nodo = raiz
    stringSugerida = ""  # guarda os caracteres já encontrados para sugerir os próximos
    
    if not raiz.filhos:   # se o nodo raiz não tiver nenhum filho, trivial, retorna falso
        return False, []
    for char in palavra:
        charNaoEncontrado = True
        for filho in nodo.filhos:
            if filho.char == char:
                charNaoEncontrado = False   # assinala que o char foi encontrado
                stringSugerida = stringSugerida+char
                nodo = filho   # passa iteração para o nodo filho
                break
        
        if charNaoEncontrado:   # chama a função que procura sugestões e retorna a tupla (False, lista de sugestões)
            return False, sugereStrings(select, dados, nodo, stringSugerida, 15)
    
    # Caso passe por todos os caracteres sem retornar false, então significa que a palavra foi encontrada
    # Resta saber se aquele nodo é um nodo final com um índice associado
    if nodo.pFinalizada:
        return True, nodo.indices
    else:
        return False, sugereStrings(select, dados, nodo, stringSugerida, 15)

#### Funções utilizadas para sugerir buscas

In [23]:
def sugereStrings(select, dados, nodo, string, maxSugestoes):   # onde select igual a 1 significa que se procura um artista e False, música
        listaDeSugestoes = []   # cria lista vazia para adicionarmos as sugestões
        procuraSugestoes(nodo, string, listaDeSugestoes)   # chama a função que percorre a trie procurando sugestões
        
        
        itera = copy.deepcopy(listaDeSugestoes)   # utilizado para iteração na lista que está sendo modificada enquanto o loop itera
        
        # substitui os indices pelos títulos
        for indice in itera:
            listaDeSugestoes.remove(indice)   # remove o índice 
            if select:
                listaDeSugestoes.append(dados[indice]['Artista']) 
            else:
                listaDeSugestoes.append(dados[indice]['Titulo']) 
            
        
        listaDeSugestoes = sorted(listaDeSugestoes)   # ordena a lista alfabeticamente
        return listaDeSugestoes[:maxSugestoes]   # retorna apenas os n elementos da lista definidos por maxSugestoes

    
# função que dado um nodo, uma parte correta de uma string (que existe na TRIE), uma lista de sugestoes, retorna uma lista de sugestões
def procuraSugestoes(nodo, string, listaDeSugestoes):
        
        if nodo.pFinalizada:   
            listaDeSugestoes.append(nodo.indices[0])   # adiciona nova sugestão na lista
        
        for filho in nodo.filhos:
            
            procuraSugestoes(filho, string+filho.char, listaDeSugestoes)   # recursão para cada filho 
            if filho.pFinalizada:   
                listaDeSugestoes.append(filho.indices[0])   # adiciona nova sugestão na lista
                
        return        

#### Função auxiliar para adicionar artista levando-se em consideração a retirada do possível "feat" da string

In [7]:
# A função checa se a string passada como parâmetro contém uma sinalização de featuring para evitar que se crie um "artista novo"
def adicionaArtista(raiz, artista, indice):
    
    if " Featuring" in artista:   # Se a música tiver um featuring, deve-se retirá-lo
        indexFeat = artista.find(" Featuring")
        counter = 0
        artistaNewString = ""
        
        # como artista = [:indexFeat] dá erro de sintaxe, precisa-se iterar e construir outra string 
        # um loop for com contador (e break) é necessário nesse caso para poder iterar pela sting 
        for char in artista:
            if counter == indexFeat:
                break
            artistaNewString = artistaNewString + char
            counter += 1

        artista = artistaNewString   # atualiza nome do artista     

    addString(raiz, artista, indice)
    

## Início do código em si

#### Leitura dos dados filtrados por ano

In [8]:
import pickle
import os

#Faz a leitura do arquivo de registros
with (open('teste.bin', 'rb')) as openfile:
    leitura = pickle.load(openfile)

#### Criação da TRIE para músicas e da TRIE para artistas

In [9]:
raizMusicas = TrieNode('@')   # raiz da trie de músicas 
raizArtistas = TrieNode('@')  # raiz da trie de artistas

indice = 0   # endereço da música no arquivo

for x in leitura:  
    # o str é necessário para os casos em que a música ou artista são apenas números e interpretados com inteiros
    addString(raizMusicas, str(x['Titulo']), indice)   
    adicionaArtista(raizArtistas, str(x['Artista']), indice)
    indice += 1

##### Testes com as TRIE criadas

In [10]:
# sendo o parâmetro 0 utilizado para sinalizar que a busca é por músicas
print(findString(0, leitura, raizMusicas, "All Of Me"), "\n")
print(findString(0, leitura, raizArtistas, "Train"), "\n")
print(findString(0, leitura, raizArtistas, "Train")[1], "\n")

(True, [10800]) 

(True, [7948, 8090, 8201, 8564, 8624, 9977, 10000, 10095, 10318, 10413, 10471]) 

[7948, 8090, 8201, 8564, 8624, 9977, 10000, 10095, 10318, 10413, 10471] 



In [11]:
print(leitura[10800])

{'Artista': 'John Legend', 'Titulo': 'All Of Me', 'Ano': 2014, 'Pontos': 4069, 'Peak': 100, 'Semanas': 59}


#### Salvando os dados das TRIE em arquivos binários

In [12]:
filename = "trieMusicas.bin"
with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
    pickle.dump(raizMusicas, openfile)
    
filename = "trieArtistas.bin"
with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
    pickle.dump(raizArtistas, openfile)

#### Testando arquivos salvos:

In [13]:
filename = "trieMusicas.bin"

if os.path.exists(filename): #confere se o arquivo existe
    with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
        trieMusicas = pickle.load(openfile)
        
filename = "trieArtistas.bin"
if os.path.exists(filename): #confere se o arquivo existe
    with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
        trieArtistas = pickle.load(openfile)
        
print(findString(0, leitura, trieMusicas, "All Of Me"), "\n")
print(findString(0, leitura, trieMusicas, "ALL OF ME"), "\n")
print(findString(0, leitura, trieMusicas, "Àll Öf Mé"), "\n")
print(findString(1, leitura, trieArtistas, "Train")[1], "\n")

(True, [10800]) 

(True, [10800]) 

(True, [10800]) 

[7948, 8090, 8201, 8564, 8624, 9977, 10000, 10095, 10318, 10413, 10471] 



In [25]:
(encontrou, sugestoes) = findString(1, leitura, trieArtistas, "Katy Perry Featuring Kanye West")
print(encontrou)
print(sugestoes)


False
['Katy Perry']
